# Spatial Planning Optimization for DRC 30 x 30 Target

This notebook executes the spatial planning optimization for recommending new areas to be protected in the DRC under the 30 x 30 initiative targets.

### Import Packages
This part loads necessary packages to execute the code. 

In [ ]:
from scripts import utils
import yaml

### Load Scenario Configurations
In this section, the configuration file corresponding to the scenario being optimized is loaded. To run the model for any other scenario, replace the path to the configuration file in the following cell.

In [ ]:
with open("configs/config_mixed.yml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
drc_df = utils.load_data(config, preprocess = True)

### Calculate the Multi-Objective Score

In [ ]:
drc_df["objective_score"] = utils.multi_c_obj(drc_df, config)

### Visualize the Multi-Objective Score

In [ ]:
utils.visualize(drc_df,
                "objective_score",
                title = "Assigned Objective Score Distribution"
                )

### Identify Candidate Planning Units for Protection 

In [ ]:
drc_df = utils.iden_units(drc_df, config)

In [ ]:
# Visualize Candidate Planning Units
utils.visualize(drc_df,
                "candidate",
                colorbar=False,
                title = "Candidate Spatial Units for Protection"
                )

### Optimize the Model 

Assumptions:
- Each planning unit can only be protected or not protected; there is no possibility for partial protection. 
- The goal of the optimization is to select planning units that maximize the multi-objective score.
- A bonus is added for selection planning units that are connected to existing or other selected planning units.
- The model maximzes the protection with a constraint to not go beyond 30% of the land.
- The model assumes an even cost of protecting new land, and only considers spatial attributes for optimization.

In [ ]:
# Model Setup
model, x = utils.setup_model(drc_df, 
                             protection_frac = 0.30,
                             connectivity = True
                             )

In [ ]:
# Solve the Model
drc_df = utils.solve_model(model, x, drc_df, threads=20)

### Visualize Proposed Units for Protection

In [ ]:
# Validate percentage of area selected to be protected
used_area = float((drc_df["area"] * drc_df["selected"]).sum())
cand_area_total = float(drc_df.loc[drc_df["candidate"], "area"].sum())
print(f"Protected area: {used_area:.2f} ({used_area/cand_area_total:.2%})")

In [ ]:
# Visualize selected units to protect
utils.visualize(drc_df,
                "selected",
                colorbar=False,
                file_name=f"{config['name']}.png",
                title=config["name"] + " - Selected Areas to Protect"
                )

In [ ]:
# Export selected units to GeoTIFF
utils.export_to_geotiff(
    drc_df,
    "selected",
    f"{config['name']}.tif"
)

In [ ]:
# Visualize existing protected areas
utils.visualize(drc_df,
                "protected",
                colorbar=False,
                file_name=f"{config['name']} - Existing Protected Areas.png",
                title=config["name"] + " - Existing Protected Areas"
                )

In [ ]:
# Export existing protected areas to GeoTIFF
utils.export_to_geotiff(
    drc_df,
    "protected",
    f"{config['name']} - Existing Protected Areas.tif"
)